In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import timescaledb_model as tsdb
from analyze import get_files_infos_df, store_file, get_files_infos_windows_df
from tqdm import tqdm 
from utils import multi_read_df_from_paths
from constant import DATA_PATH
import os

Logs of timescaledb_model go to ../data/bourse.log


In [4]:
files_info_path = os.path.join(DATA_PATH, 'files_infos.pkl')
files_infos_df = get_files_infos_df()
# files_infos_df = get_files_infos_windows_df()  # for windows
files_infos_df.to_pickle(files_info_path)

## Files Dataframe - Filtering
1 hour for each Last day of Each month

In [5]:
files_infos_df = get_files_infos_df(files_info_path)
# files_infos_df = get_files_infos_windows_df(files_info_path) #for windows

In [6]:
np.random.seed(0)
dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
file_infos_df_filtered = files_infos_df[files_infos_df["date"].isin(dates_to_take)]

/var/folders/lb/9z4xr9hd1013w3_cbsw1p6600000gn/T/ipykernel_34634/525646213.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]


In [7]:
file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour
file_infos_df_filtered = file_infos_df_filtered.groupby([file_infos_df_filtered["date"], file_infos_df_filtered["hour"], file_infos_df_filtered["market"]]).first().reset_index()

/var/folders/lb/9z4xr9hd1013w3_cbsw1p6600000gn/T/ipykernel_34634/3272020381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour


In [8]:
# dfs = multi_read_df_from_paths(list(file_infos_df_filtered["path"]))

## PEA PME - Symbol NF handling

### Check if every files from PEA PME have the same prefixes

In [9]:
dfs = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "peapme"] ["path"]))

 22%|██▏       | 71/324 [00:00<00:00, 705.55it/s]

100%|██████████| 324/324 [00:02<00:00, 149.59it/s]


In [10]:
def get_prefix(df: pd.DataFrame, prefix_function = lambda x : x[0:3]) -> set:
    df["prefix"] = df["symbol"].apply(prefix_function)
    return set(df["prefix"].unique())


In [11]:
# prefixes = set()
# for i, df in enumerate(dfs):
#     current_prefixes = get_prefix(df)
#     if not current_prefixes.issubset(prefixes):
#         prefixes = prefixes.union(current_prefixes)
#         # print(current_prefixes, i)
# prefixes

In [12]:
map_prefix_to_symbol_nf = {
    "1rP": lambda x: x[3:].split("_")[0] if len(x) != 15 else np.nan,
    "1rA": lambda x: x[3:],
    "1rE": lambda x: x[4:],
    "FF1": lambda x: x.split("_")[1][0:],
}

In [13]:
def filter_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df["last"] != 0]
    return df

def add_symbol_nf_column(df: pd.DataFrame, map_prefix_to_symbol_nf: dict) -> pd.DataFrame:
    df["prefix"] = df["symbol"].apply(lambda x: x[0:3])
    df["symbol_nf"] = df.apply(
        lambda x: map_prefix_to_symbol_nf.get(x["prefix"], lambda _: x["symbol"])(x["symbol"]),
        axis=1,
    )
    return df

In [14]:
df = filter_df(dfs[0])
df = add_symbol_nf_column(df, map_prefix_to_symbol_nf)
symbol_nfs = set(df["symbol_nf"].unique())

In [15]:
df["len"] = df["symbol_nf"].apply(len)
df.sort_values("len", ascending=False)

,last,volume,symbol,name,last_suffix,prefix,symbol_nf,len
symbol,,,,,,,,
1rPALGAUXX,0.115,0,1rPALGAUXX,GAUSSIN,c,1rP,ALGAUXX,7
1rEPALMIL,18.950,0,1rEPALMIL,1000MERCIS,c,1rE,ALMIL,5
1rPMLCLI,2.020,0,1rPMLCLI,MAISON CLIO,c,1rP,MLCLI,5
1rPMEDCL,13.300,4195,1rPMEDCL,MEDINCELL,NaN,1rP,MEDCL,5
1rPALMED,6.900,0,1rPALMED,MEDICREA INTERN.,c,1rP,ALMED,5
...,...,...,...,...,...,...,...,...
1rPIVA,12.740,6925,1rPIVA,INVENTIVA,NaN,1rP,IVA,3
1rPHF,4.950,2170,1rPHF,HF COMPANY,NaN,1rP,HF,2
1rPGV,0.804,792116,1rPGV,GENOMIC VISION,c,1rP,GV,2


In [23]:
df["last_suffix"].value_counts()

last_suffix
c    297
s      7
Name: count, dtype: int64

## CompA - Symbol NF handling